# GPT For Bigram Language Model

Here we build a complete working Generative Pre-trained Transformer (GPT) model from scratch capable of generating text in the style of Shakespeare.

## 1. Setup and Hyperparameters

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [18]:
batch_size = 16#how many independent sequences will we process in parallel
block_size = 32# what is the maximum context length for predictions
max_iters = 5000# how many training iterations
eval_interval = 100# evaluate the loss every eval_interval iterations
learning_rate = 1e-3# learning rate for optimization
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200# how many batches to use for evaluation
n_embd = 64# the embedding dimension
n_head = 4# the number of heads in the multi-head attention model
n_layer = 4# the number of layers in the transformer
dropout = 0.0# the dropout rate

## 2. Data Loading and Tokenization

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
# encoder: take a string, output a list of integers
def encode(s): return [stoi[c] for c in s]
# decoder: take a list of integers, output a string
def decode(l): return ''.join([itos[i] for i in l])
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## 3. Data Batching and Loss Estimation


### Data Loading Function `(get_batch)`


In [5]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Loss Estimation Function `(estimate_loss)`

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}# results will be collected here
    model.eval()# set the model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)# tensor to hold the loss values
        for k in range(eval_iters):
            X, Y = get_batch(split)# get a batch of data
            logits, loss = model(X, Y)# forward pass through the model
            losses[k] = loss.item()# store the loss value
        out[split] = losses.mean()# compute the mean loss for this split
    model.train()# set the model back to training mode
    return out

## 4. Transformer Model Architecture

### `Head` Module: A Single Head of Self-Attention

- `__init__:`
    - It creates three `nn.Linear` layers to produce the Query, Key, and Value vectors from the input with no bias term.
    - `self.register_buffer('tril', ...)`: This creates a lower-triangular matrix of ones called `tril`. A buffer is a parameter that is part of the model's state but is not trained by the optimizer. This mask is essential for creating a decoder-style Transformer that can only attend to previous tokens in the sequence.

- `forward:`
    - It calculates the attention scores (`wei`) by taking the dot product of queries and keys and scaling by the square root of the channel dimension (C**-0.5).
    - `wei.masked_fill(...)`: This applies the causal `mask`. It prevents tokens from seeing future tokens by setting the attention scores for future positions to negative infinity.
    - `F.softmax`: Converts the scores into a probability distribution.
    - `wei @ v`: Performs the weighted aggregation of the value vectors to produce the output of the attention head.

In [7]:
#One head of self attention
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)# key projection
        self.query = nn.Linear(n_embd, head_size, bias = False)# query projection
        self.value = nn.Linear(n_embd, head_size, bias = False)# value projection
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))# lower triangular matrix for masking
        self.dropout = nn.Dropout(dropout)# dropout layer
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,head_size) @ (B,head_size,T) -> (B,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))# apply the mask to ensure causality
        wei = F.softmax(wei, dim=-1)# softmax to get attention weights
        wei = self.dropout(wei)# apply dropout to the attention weights
        v = self.value(x) # (B,T,head_size)
        out = wei @ v # (B,T,T) @ (B,T,head_size) -> (B,T,head_size)
        return out

### `MultiHeadAttention` Module

This module combines multiple Heads to run in parallel.
- `__init__`: It creates `num_heads` instances of our ``Head` class and stores them in an `nn.ModuleList`. It also has a final projection layer (`self.proj`) that maps the concatenated head outputs back to the original embedding dimension.

- `forward`: It runs each head on the input x concatenates their outputs along the channel dimension and then passes the result through the projection layer.

In [8]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):
        super().__init__()# initialize the multi-head attention module
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])# create multiple heads
        self.proj = nn.Linear(n_embd, n_embd)# final projection layer
        self.dropout = nn.Dropout(dropout)# dropout layer

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)# concatenate the outputs of all heads
        out = self.dropout(self.proj(out))# final projection and dropout
        return out

### `FeedForward` Module

In [9]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### Block Module: The Transformer Block

In [10]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)# self-attention
        self.ffwd = FeedFoward(n_embd)# feed-forward network
        self.ln1 = nn.LayerNorm(n_embd)# layer norm 1
        self.ln2 = nn.LayerNorm(n_embd)# layer norm 2

    def forward(self, x):
        x = x + self.sa(self.ln1(x))# residual connection around self-attention
        x = x + self.ffwd(self.ln2(x))# residual connection around feed-forward network
        return x

### `BigramLanguageModel`

In [20]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)# position embeddings
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)])# stack of transformer blocks
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)# language model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(
            torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape# batch size, time steps, vocab size
            logits = logits.view(B*T, C)# reshape logits to (B*T, C)
            targets = targets.view(B*T)# reshape targets to (B*T)
            loss = F.cross_entropy(logits, targets)# compute the cross-entropy loss

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



0.209729 M parameters


In [22]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)  # reset the gradients
    loss.backward()  # backpropagate to compute gradients
    optimizer.step()  # perform a parameter update

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

step 0: train loss 1.6574, val loss 1.8160
step 100: train loss 1.6566, val loss 1.8172
step 200: train loss 1.6344, val loss 1.8100
step 300: train loss 1.6545, val loss 1.8224
step 400: train loss 1.6500, val loss 1.8052
step 500: train loss 1.6421, val loss 1.7940
step 600: train loss 1.6368, val loss 1.8067
step 700: train loss 1.6360, val loss 1.8089
step 800: train loss 1.6324, val loss 1.7921
step 900: train loss 1.6272, val loss 1.7870
step 1000: train loss 1.6230, val loss 1.7834
step 1100: train loss 1.6311, val loss 1.7822
step 1200: train loss 1.6212, val loss 1.7872
step 1300: train loss 1.6261, val loss 1.7828
step 1400: train loss 1.6129, val loss 1.7790
step 1500: train loss 1.6141, val loss 1.7829
step 1600: train loss 1.6014, val loss 1.7814
step 1700: train loss 1.6109, val loss 1.7812
step 1800: train loss 1.6050, val loss 1.7684
step 1900: train loss 1.6014, val loss 1.7809
step 2000: train loss 1.5982, val loss 1.7747
step 2100: train loss 1.5979, val loss 1.7683


In [24]:
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


Have no know them, which abitory,
To my peeer! suffs of, Leazen: Say, dead not for we havishmasion: dear homeld be murder;
As theye there breat and pleaded own,
being breathe, my pare mock, when better, Leozen of him Take.
I did know this fellow: flady, yea!
Stand o' the woman: and he silks as know him one loves?

QUEEN MARTIUS:
Would but where is freals of cheacher no courts
I in the which and shosest unto what see be me to no vicievee to chance:
Since uncanswer; he humeting's kneess sweet body,.
O, here not on earth, the vice of for thoughtsing this: and if thy bidition by my arm repolling;
I pase apastage up out that his shall truke cule I earth,
Dearing what give be leverent, lived 
BUCK:
Could forth my faught.

DORD:
Wout took that is the not; or what is storry shall his herm,
Here loves as whan to stortake, it this
say abless peace celset me; picit
feed, woo; and thind will cause a vine be leforne busin and some
All from verbilood: asay' as from their scan to snift;
Now'tings sa